In [1]:
import os, glob
import numpy as np
import librosa
import pickle
import madmom
import essentia.standard as es
from tqdm import tqdm

def load_pickle(filepath):
    with open(filepath, "rb") as f:
        json_data = pickle.load(f)
    return json_data

mp3_path = "aist_dataset/mp3"
mp3_files = sorted(glob.glob(os.path.join(mp3_path, "*.mp3")))

plp_bpms = []
madmom_bpms = []
essentia_bpms = []
percival_bpms = []
madmom_cnn_bpms = []
tempo_cnn_bpms = []

# 1. PLP-based tempo (via librosa)
def estimate_bpm_plp(audio_file, sr=22050):
    
    y, sr = librosa.load(audio_file, sr=sr)
    # Onset strength envelope
    oenv = librosa.onset.onset_strength(y=y, sr=sr)
    # PLP curve
    plp = librosa.beat.plp(onset_envelope=oenv, sr=sr)
    # Convert PLP (Hz) to BPM
    bpm = 60.0 * plp.mean()
    return bpm

# 2. DL baseline using Madmom (RNN + TempoEstimation)
# Böck, S., Krebs, F., & Widmer, G. (2015). Accurate Tempo Estimation Based on Recurrent Neural Networks and Resonating Comb Filters. ISMIR 2015.
rnn_beat = madmom.features.beats.RNNBeatProcessor()
tempo_proc = madmom.features.tempo.TempoEstimationProcessor(fps=100, method='comb')


def estimate_bpm_madmom(audio_file):
    
    act = rnn_beat(audio_file)  # tempo search range is [30, 285] BPM by default
    tempo_arr = tempo_proc(act)
    bpm = tempo_arr[0][0]  # most likely tempo in BPM
    return bpm

# 3. Classical baseline via Essentia
# Zapata, J., Davies, M. E. P., & Serra, X. (2014). Multi-Feature Beat Tracker. IEEE/ACM Transactions on Audio, Speech, and Language Processing, 22(4), 816–825.
rhythm_extractor = es.RhythmExtractor2013(method="multifeature")

def estimate_bpm_essentia(audio_file):
    
    loader = es.MonoLoader(filename=audio_file)
    audio = loader()
    bpm, _, _, _, _ = rhythm_extractor(audio)   # tempo search range is [40, 208] BPM by default    
    return bpm

# 4. Percival's method via Essentia
# Percival, G., & Tzanetakis, G. (2014). Streamlined Tempo Estimation Based on Autocorrelation and Cross-Correlation with Pulses. IEEE/ACM Transactions on Audio, Speech, and Language Processing, 22(12), 1765–1776.
percival_estimator = es.PercivalBpmEstimator(minBPM=60, maxBPM=140)

def estimate_bpm_percival(audio_file):
    
    loader = es.MonoLoader(filename=audio_file)
    audio = loader()
    bpm = percival_estimator(audio)     # tempo search range is [50, 210] BPM by default        
    return bpm

# Schreiber & Müller (ISMIR 2018)
def estimate_bpm_tempocnn(audio_file, model_path="/itf-fi-ml/home/sagardu/aist_tempo_est/deepsquare-k16-3.pb", sr=11025):
    
    # Load audio at the expected rate
    audio = es.MonoLoader(filename=audio_file, sampleRate=sr, resampleQuality=4)()    
    # Load CNN model
    model = es.TempoCNN(graphFilename=model_path)
    global_tempo, local_tempo, local_tempo_probabilities = model(audio)
    return float(global_tempo)


for f in tqdm(mp3_files):
    # print(f"Processing {os.path.basename(f)}")
    # plp_bpms.append(estimate_bpm_plp(f))
    
    madmom_bpms.append(estimate_bpm_madmom(f))
    essentia_bpms.append(estimate_bpm_essentia(f))
    
    bpm_percival = estimate_bpm_percival(f)
    percival_bpms.append(bpm_percival)
    tempo_cnn_bpms.append(estimate_bpm_tempocnn(f))
    

madmom_bpms = np.array(madmom_bpms)
essentia_bpms = np.array(essentia_bpms)
percival_bpms = np.array(percival_bpms)
cnn_bpms = np.array(tempo_cnn_bpms)


2025-10-06 14:47:52.670495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-10-06 14:47:52.670542: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[   INFO   ] MusicExtractorSVM: no classifier models were configured by default
2025-10-06 14:47:53.819897: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-06 14:47:53.828218: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2025-10-06 14:47:5

In [2]:
def compute_dts(
    ref_bpm,
    estimated_bpm,
    tau=0.06,
    mode="one"
):

    ref_bpm = np.asarray(ref_bpm, dtype=float)

    # select a single estimate per index if needed
    if mode == "many":
        # estimated_bpm is e.g. [(b1, b2,...), (c1, c2,...), ...]
        chosen = np.array([
            min(cands, key=lambda b: abs(b - ref_bpm[i]))
            for i, cands in enumerate(estimated_bpm)
        ], dtype=float)
    elif mode == "one":
        chosen = np.asarray(estimated_bpm, dtype=float)
    else:
        raise ValueError(f"Unknown mode: {mode!r}. Use 'one' or 'many'.")

    # now compute the DTS exactly as before
    e = np.log2(chosen / ref_bpm)
    # distance from nearest of -1, 0, +1
    d = np.abs(e[:, None] - np.array([-1.0, 0.0, 1.0])).min(axis=1)
    # clip by tolerance and convert to score
    d_clip = np.minimum(d, tau)
    dts    = 1.0 - d_clip / tau

    accuracy = (dts > 0.0).mean() * 100
    
    # hits ----------------------------------------------------------
    hit_mask = dts > 0.0          # inside ±tau band
    hit_idx = np.nonzero(hit_mask)[0]
    ref_hit_bpm = ref_bpm[hit_idx]
    
    return dts, e, d, accuracy


In [3]:
ref_dict = {    "mBR0": 80,    "mBR1": 90,    "mBR2": 100,    "mBR3": 110,    "mBR4": 120,
    "mBR5": 130,    "mPO0": 80,    "mPO1": 90,    "mPO2": 100,    "mPO3": 110,    "mPO4": 120,
    "mPO5": 130,    "mLO0": 80,    "mLO1": 90,    "mLO2": 100,    "mLO3": 110,    "mLO4": 120,
    "mLO5": 130,    "mMH0": 80,    "mMH1": 90,    "mMH2": 100,    "mMH3": 110,    "mMH4": 120,
    "mMH5": 130,    "mLH0": 80,    "mLH1": 90,    "mLH2": 100,    "mLH3": 110,    "mLH4": 120,
    "mLH5": 130,    "mHO0": 110,    "mHO1": 115,    "mHO2": 120,    "mHO3": 125,    "mHO4": 130,
    "mHO5": 135,    "mWA0": 80,    "mWA1": 90,    "mWA2": 100,    "mWA3": 110,    "mWA4": 120,
    "mWA5": 130,    "mKR0": 80,    "mKR1": 90,    "mKR2": 100,    "mKR3": 110,    "mKR4": 120,
    "mKR5": 130,    "mJS0": 80,    "mJS1": 90,    "mJS2": 100,    "mJS3": 110,    "mJS4": 120,
    "mJS5": 130,    "mJB0": 80,    "mJB1": 90,    "mJB2": 100,    "mJB3": 110,    "mJB4": 120,    "mJB5": 130
}

# Build reference array
file_ids = [os.path.splitext(os.path.basename(f))[0] for f in mp3_files]
ref = np.array([ref_dict[fid] for fid in file_ids])

ta = 0.05

# _, _, _, accuracy_plp  = compute_dts(ref, plp_bpms, tau= ta, mode="one")
_, _, _, accuracy_madmom  = compute_dts(ref, madmom_bpms, tau= ta, mode="one")
_, _, _, accuracy_essentia  = compute_dts(ref, essentia_bpms, tau= ta, mode="one")
_, _, _, accuracy_percival  = compute_dts(ref, percival_bpms, tau= ta, mode="one")
_, _, _, accuracy_cnn  = compute_dts(ref, cnn_bpms, tau= ta, mode="one")


print("=== Tempo Estimation Accuracy with DTS (±3.5%) ===")
print(f"Accuracy Essentia: {accuracy_essentia:.2f}%")
print(f"Accuracy Madmom: {accuracy_madmom:.2f}%")
print(f"Accuracy Percival: {accuracy_percival:.2f}%")
print(f"Accuracy TempoCNN: {accuracy_cnn:.2f}%")



=== Tempo Estimation Accuracy with DTS (±3.5%) ===
Accuracy Essentia: 98.33%
Accuracy Madmom: 100.00%
Accuracy Percival: 15.00%
Accuracy TempoCNN: 98.33%


In [4]:
import numpy as np

def tempo_accuracy(ref, est, tol=0.035):
    """
    Compute Accuracy1 and Accuracy2 for tempo estimation.
    
    Parameters
    ----------
    ref : array-like
        Reference tempi (BPM)
    est : array-like
        Estimated tempi (BPM)
    tol : float
        Relative tolerance (default 0.04 = ±4%)
    """
    ref = np.asarray(ref)
    est = np.asarray(est)

    # --- Accuracy1: strict match within tol ---
    correct1 = np.abs(est - ref) / ref <= tol
    acc1 = np.mean(correct1)

    # --- Accuracy2: allow octave errors (x0.5, x2) ---
    correct2 = []
    for r, e in zip(ref, est):
        rel_err = [abs(e - r) / r,
                   abs(2*e - r) / r,
                   abs(0.5*e - r) / r]
        correct2.append(np.min(rel_err) <= tol)
    acc2 = np.mean(correct2)

    return acc1, acc2


In [5]:
# acc1_plp, acc2_plp = tempo_accuracy(ref, plp_bpms, tol=0.04)
acc1_madmom, acc2_madmom = tempo_accuracy(ref, madmom_bpms, tol=0.04)
acc1_ess, acc2_ess = tempo_accuracy(ref, essentia_bpms, tol=0.04)
acc1_percival, acc2_percival = tempo_accuracy(ref, percival_bpms, tol=0.04)
acc1_cnn, acc2_cnn = tempo_accuracy(ref, cnn_bpms, tol=0.04)

print("=== Tempo Estimation Accuracy with Accuracy1 and Accuracy2 (±3.5%) ===")

print("Madmom - Acc1:", acc1_madmom, "Acc2:", acc2_madmom)
print("Essentia - Acc1:", acc1_ess, "Acc2:", acc2_ess)
print("Percival - Acc1:", acc1_percival, "Acc2:", acc2_percival)
print("TempoCNN - Acc1:", acc1_cnn, "Acc2:", acc2_cnn)


=== Tempo Estimation Accuracy with Accuracy1 and Accuracy2 (±3.5%) ===
Madmom - Acc1: 0.9333333333333333 Acc2: 1.0
Essentia - Acc1: 0.8666666666666667 Acc2: 0.9833333333333333
Percival - Acc1: 0.15 Acc2: 0.15
TempoCNN - Acc1: 0.9833333333333333 Acc2: 0.9833333333333333


In [6]:
a, b = 60, 140
root = "/itf-fi-ml/home/sagardu/aist_tempo_est/saved_result_adaptive"
pth_pos = f"{root}/tempo_{a}_{b}/pos"

for fname in os.listdir(pth_pos):
    if fname == f"adaptv_Bhandfoot_y_zero_uni_W5_H2.5_{a}_{b}.pkl":
    
        print(f"Processing {fname}...")
        tag = fname.split("_zero_uni")[0]
        data = load_pickle(f"{pth_pos}/{fname}")
        ref  = data["music_tempo"].to_numpy()

# data["bpm_median"]  # dance tempo estimates
# data["music_id"]    # music IDs corresponding to the estimates

Processing adaptv_Bhandfoot_y_zero_uni_W5_H2.5_60_140.pkl...


In [7]:
# Get music IDs (without extension)
music_ids = [os.path.splitext(os.path.basename(f))[0] for f in mp3_files]

# Build dicts for lookup
plp_dict = dict(zip(music_ids, plp_bpms))
madmom_dict = dict(zip(music_ids, madmom_bpms))
essentia_dict = dict(zip(music_ids, essentia_bpms))
percival_dict = dict(zip(music_ids, percival_bpms))
cnn_dict = dict(zip(music_ids, cnn_bpms))

In [8]:
dance_ids = data["music_id"]        # length ~1300
dance_bpms = data["bpm_mode"]     # length ~1300

# For each dance entry, fetch corresponding MIR tempo
# plp_for_dance = np.array([plp_dict[mid] for mid in dance_ids])
madmom_for_dance = np.array([madmom_dict[mid] for mid in dance_ids])
essentia_for_dance = np.array([essentia_dict[mid] for mid in dance_ids])
percival_for_dance = np.array([percival_dict[mid] for mid in dance_ids])
cnn_for_dance = np.array([cnn_dict[mid] for mid in dance_ids])


In [9]:
def agreement_rate(dance_est, mir_est, tol=0.04):
    dance_est, mir_est = np.asarray(dance_est), np.asarray(mir_est)
    # Strict agreement (Acc1)
    acc1 = np.mean(np.abs(dance_est - mir_est) / mir_est <= tol)
    # Agreement with octave flexibility (Acc2)
    acc2 = np.mean([
        min(abs(d - m) / m, abs(2*d - m) / m, abs(0.5*d - m) / m) <= tol
        for d, m in zip(dance_est, mir_est)
    ])
    return acc1, acc2

def confusion_levels(est, ref, tol=0.04):
    categories = []
    for e, r in zip(est, ref):
        if abs(e - r) / r <= tol:
            categories.append("on-time")
        elif abs(e - 0.5*r) / r <= tol:
            categories.append("half-time")
        elif abs(e - 2*r) / r <= tol:
            categories.append("double-time")
        else:
            categories.append("off-grid")
    return categories

In [10]:
# acc1_plp, acc2_plp = agreement_rate(dance_bpms, plp_for_dance)
acc1_madmom, acc2_madmom = agreement_rate(dance_bpms, madmom_for_dance)
acc1_ess, acc2_ess = agreement_rate(dance_bpms, essentia_for_dance)
acc1_percival, acc2_percival = agreement_rate(dance_bpms, np.array([percival_dict[mid] for mid in dance_ids]))
acc1_cnn, acc2_cnn = agreement_rate(dance_bpms, cnn_for_dance)


print("\n=== Agreement with Audio (Dance vs MIR) ===")

print(f"Madmom  → Acc1: {acc1_madmom:.3f}, Acc2: {acc2_madmom:.3f}")
print(f"Essentia→ Acc1: {acc1_ess:.3f}, Acc2: {acc2_ess:.3f}")
print(f"Percival→ Acc1: {acc1_percival:.3f}, Acc2: {acc2_percival:.3f}")
print(f"TempoCNN→ Acc1: {acc1_cnn:.3f}, Acc2: {acc2_cnn:.3f}")



=== Agreement with Audio (Dance vs MIR) ===
Madmom  → Acc1: 0.412, Acc2: 0.462
Essentia→ Acc1: 0.362, Acc2: 0.444
Percival→ Acc1: 0.094, Acc2: 0.094
TempoCNN→ Acc1: 0.411, Acc2: 0.440


In [11]:
ref_dict_full = {mid: bpm for mid, bpm in ref_dict.items()}  # your dict
ref_for_dance = np.array([ref_dict_full[mid] for mid in dance_ids])

# Confusion analysis
dance_conf = confusion_levels(dance_bpms, ref_for_dance)
madmom_conf = confusion_levels(madmom_for_dance, ref_for_dance)
ess_conf   = confusion_levels(essentia_for_dance, ref_for_dance)
percival_conf = confusion_levels(percival_for_dance, ref_for_dance)
tempocnn_conf = confusion_levels(cnn_for_dance, ref_for_dance)


In [ ]:
from collections import Counter

def print_confusion(label, conf_list):
    counts = Counter(conf_list)
    total = sum(counts.values())
    print(f"\n{label} confusion:")
    for cat in ["half-time", "on-time", "double-time", "off-grid"]:
        print(f"  {cat:10s}: {counts[cat]:4d} ({counts[cat]/total:.2%})")

print("\n=== Confusion Across Metrical Levels ===")
print_confusion("Dance", dance_conf)
print_confusion("Madmom", madmom_conf)
print_confusion("Essentia", ess_conf)
print_confusion("Percival", percival_conf)
print_confusion("TempoCNN", tempocnn_conf)


# By Genre

In [13]:
import json

with open("genre_symbols_mapping.json", "r") as f:
    genre_symbols = json.load(f)

with open("genreID_count_mapping.json", "r") as f:
    genre_counts = json.load(f)


In [14]:
dance_bpms   = data["bpm_median"].to_numpy()
dance_ids    = data["music_id"].to_numpy()
dance_genres = data["dance_genre"].to_numpy()

ref_for_dance = np.array([ref_dict[mid] for mid in dance_ids])


In [15]:
from collections import defaultdict

agreement_by_genre = defaultdict(dict)

for g in np.unique(dance_genres):
    mask = dance_genres == g
    ref_g   = ref_for_dance[mask]
    dance_g = dance_bpms[mask]
    
    # MIR-aligned estimates for this subset
 
    madmom_g  = madmom_for_dance[mask]
    ess_g     = essentia_for_dance[mask]
    percival_g= percival_for_dance[mask]

    agreement_by_genre[g]["Madmom"]   = agreement_rate(dance_g, madmom_g)
    agreement_by_genre[g]["Essentia"] = agreement_rate(dance_g, ess_g)
    agreement_by_genre[g]["Percival"] = agreement_rate(dance_g, percival_g)
    agreement_by_genre[g]["TempoCNN"] = agreement_rate(dance_g, cnn_for_dance[mask])


In [16]:
confusion_by_genre = defaultdict(dict)

for g in np.unique(dance_genres):
    mask = dance_genres == g
    ref_g   = ref_for_dance[mask]
    dance_g = dance_bpms[mask]


    madmom_g  = madmom_for_dance[mask]
    ess_g     = essentia_for_dance[mask]
    percival_g= percival_for_dance[mask]


    confusion_by_genre[g]["Dance"]    = confusion_levels(dance_g, ref_g)
    confusion_by_genre[g]["Madmom"]   = confusion_levels(madmom_g, ref_g)
    confusion_by_genre[g]["Essentia"] = confusion_levels(ess_g, ref_g)
    confusion_by_genre[g]["Percival"] = confusion_levels(percival_g, ref_g)
    confusion_by_genre[g]["TempoCNN"] = confusion_levels(cnn_for_dance[mask], ref_g)


In [17]:
print("\n=== Agreement by Genre ===")
for g in agreement_by_genre:
    gname = genre_symbols.get(g, g)
    print(f"\n{gname} ({genre_counts.get(g, 0)} sequences):")
    for method, (acc1, acc2) in agreement_by_genre[g].items():
        print(f"  {method:10s} Acc1: {acc1:.3f}, Acc2: {acc2:.3f}")

# print("\n=== Confusion by Genre ===")
# for g in confusion_by_genre:
#     gname = genre_symbols.get(g, g)
#     print(f"\n{gname} ({genre_counts.get(g, 0)} sequences):")
#     for method, conf in confusion_by_genre[g].items():
#         c = Counter(conf); N = sum(c.values())
#         print(f"  {method:10s}", end=" ")
#         for cat in ["half-time", "on-time", "double-time", "off-grid"]:
#             print(f"{cat}:{c[cat]/N:.2%}", end="  ")
#         print()



=== Agreement by Genre ===

Break (99 sequences):
  Madmom     Acc1: 0.778, Acc2: 0.788
  Essentia   Acc1: 0.677, Acc2: 0.798
  Percival   Acc1: 0.101, Acc2: 0.101
  TempoCNN   Acc1: 0.778, Acc2: 0.788

House (136 sequences):
  Madmom     Acc1: 0.978, Acc2: 0.978
  Essentia   Acc1: 0.978, Acc2: 0.978
  Percival   Acc1: 0.000, Acc2: 0.000
  TempoCNN   Acc1: 0.978, Acc2: 0.978

Ballet Jazz (136 sequences):
  Madmom     Acc1: 0.169, Acc2: 0.243
  Essentia   Acc1: 0.206, Acc2: 0.265
  Percival   Acc1: 0.088, Acc2: 0.088
  TempoCNN   Acc1: 0.257, Acc2: 0.272

Street Jazz (129 sequences):
  Madmom     Acc1: 0.178, Acc2: 0.186
  Essentia   Acc1: 0.178, Acc2: 0.186
  Percival   Acc1: 0.031, Acc2: 0.031
  TempoCNN   Acc1: 0.178, Acc2: 0.186

Krump (137 sequences):
  Madmom     Acc1: 0.489, Acc2: 0.511
  Essentia   Acc1: 0.453, Acc2: 0.511
  Percival   Acc1: 0.029, Acc2: 0.029
  TempoCNN   Acc1: 0.482, Acc2: 0.504

LA style Hip-hop (141 sequences):
  Madmom     Acc1: 0.560, Acc2: 0.567
  Essent